In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import time

In [2]:
nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
response = requests.get(nasa_url)
soup = BeautifulSoup(response.text, 'html.parser')
p_result = soup.find('div', class_='rollover_description_inner')
news_p = p_result.text.strip()
result = soup.find('div', class_='content_title')
news_title = result.text.strip()

In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
splint_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(splint_url)
time.sleep(1)
browser.links.find_by_partial_text('FULL IMAGE').click()
time.sleep(1)
browser.links.find_by_partial_text('more info').click()
time.sleep(1)
soup = BeautifulSoup(browser.html, 'html.parser')

img = soup.findAll('img', class_='main_image')

In [5]:
img_url = 'https://www.jpl.nasa.gov'
for image in img:
    img_url = img_url + image['src']

In [6]:
browser.quit()

In [7]:
browser = Browser('chrome', **executable_path, headless=False, incognito=True)

In [8]:
mars_twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_twitter_url)
time.sleep(1)
soup = BeautifulSoup(browser.html, 'html.parser')
first_tweet = soup.findAll('div', class_='css-1dbjc4n r-18u37iz' )

soup = BeautifulSoup(str(first_tweet), 'html.parser')

f_tweet = soup.findAll('span')
mars_weather = f_tweet[4].text.strip()

In [9]:
browser.quit()

In [ ]:
f_tweet

In [22]:
table_url = 'https://space-facts.com/mars/'
df = pd.read_html(table_url)
time.sleep(1)
mars_facts = df[0]
mars_earth = df[1]
mars_facts.rename(columns={0: '0', 1:'1'})
mars_facts = mars_facts.to_json()
mars_earth = mars_earth.to_json()

In [27]:
mars_facts

'{"0":{"0":"Equatorial Diameter:","1":"Polar Diameter:","2":"Mass:","3":"Moons:","4":"Orbit Distance:","5":"Orbit Period:","6":"Surface Temperature:","7":"First Record:","8":"Recorded By:"},"1":{"0":"6,792 km","1":"6,752 km","2":"6.39 \\u00d7 10^23 kg (0.11 Earths)","3":"2 (Phobos & Deimos)","4":"227,943,824 km (1.38 AU)","5":"687 days (1.9 years)","6":"-87 to -5 \\u00b0C","7":"2nd millennium BC","8":"Egyptian astronomers"}}'

In [12]:
hem_dicts = []
hem_titles = []
hem_url_list = []
hem_names = ['Cerberus', 'Schiaparelli', 'Syrtis', 'Valles']
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser = Browser('chrome', **executable_path, headless=False)

In [13]:
def hem_to_dict(title, url):
    d = {'title': title, 'img_url': url}
    return d

In [14]:
def get_pic_url():
    browser.links.find_by_partial_text('Sample').click()
    time.sleep(1)
    parent_window = browser.driver.current_window_handle
    all_windows = browser.driver.window_handles
    child_window = [window for window in all_windows if window != parent_window][0]
    browser.driver.switch_to.window(child_window)
    time.sleep(1)
    temp_url = browser.url
    browser.driver.close()
    browser.driver.switch_to.window(parent_window)
    return temp_url

In [15]:
for hem in hem_names:
    browser.visit(hem_url)
    time.sleep(1)
    browser.links.find_by_partial_text(hem).click()
    time.sleep(1)
    t = browser.title.split('|')
    t = t[0]
    t = t.strip()
    hem_titles.append(t)

    url = get_pic_url()
    hem_url_list.append(url)

In [16]:
browser.quit()

In [17]:
for i, x in enumerate(hem_titles):
    hem_dicts.append(hem_to_dict(x, hem_url_list[i]))

In [18]:
return_dict = { 'news_title': news_title,
                'news_p': news_p, 
                'featured_image_url': img_url, 
                'mars_weather': mars_weather,
                'mars_facts': mars_facts,
                'hem_dicts': hem_dicts
                }